# Creating Multiple Agents - A Comprehensive Guide

This guide shows you how to create multiple agents across different AI frameworks and coordinate them effectively.

## Overview

Creating multiple agents opens up powerful possibilities for:
- **Specialization**: Different agents can handle different domains or tasks
- **Collaboration**: Agents can work together, review each other's work, and iterate
- **Scalability**: Distribute workload across multiple agents
- **Robustness**: Redundancy and different approaches to solving problems

We'll cover multiple agent creation in:
1. **AutoGen** - Dynamic agent creation and coordination
2. **CrewAI** - Team-based collaboration
3. **LangGraph** - Graph-based agent orchestration
4. **OpenAI Agents SDK** - Structured multi-agent workflows

## Agent Coordination Patterns

### 1. Sequential Pattern
Agents work one after another in a pipeline:
```
Agent A → Agent B → Agent C → Result
```

### 2. Parallel Pattern  
Agents work simultaneously on different aspects:
```
        Agent A
Input →  Agent B → Combine → Result
        Agent C
```

### 3. Hierarchical Pattern
Manager agent coordinates worker agents:
```
Manager Agent
    ├── Worker Agent 1
    ├── Worker Agent 2
    └── Worker Agent 3
```

### 4. Peer-to-Peer Pattern
Agents communicate directly with each other:
```
Agent A ↔ Agent B
    ↕       ↕
Agent D ↔ Agent C
```

## 1. AutoGen - Dynamic Multi-Agent Systems

AutoGen excels at creating agents dynamically and enabling complex interactions.

### Basic Multi-Agent Setup

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv

load_dotenv(override=True)

# Create multiple specialized agents
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Research Agent
researcher = AssistantAgent(
    name="researcher",
    model_client=model_client,
    system_message="You are a research specialist. Focus on gathering and analyzing information thoroughly."
)

# Writer Agent 
writer = AssistantAgent(
    name="writer",
    model_client=model_client,
    system_message="You are a skilled writer. Take research and create clear, engaging content."
)

# Editor Agent
editor = AssistantAgent(
    name="editor",
    model_client=model_client,
    system_message="You are an editor. Review content for clarity, accuracy, and improvements."
)

print("Created three specialized agents: researcher, writer, and editor")

### Sequential Collaboration

In [ ]:
# Sequential workflow: Research → Write → Edit
async def sequential_workflow(topic):
    # Step 1: Research
    research_message = TextMessage(content=f"Please research the topic: {topic}", source="user")
    research_result = await researcher.on_messages([research_message])
    
    # Step 2: Write based on research
    write_message = TextMessage(
        content=f"Based on this research, write an article: {research_result.chat_message.content}", 
        source="user"
    )
    write_result = await writer.on_messages([write_message])
    
    # Step 3: Edit the article
    edit_message = TextMessage(
        content=f"Please edit and improve this article: {write_result.chat_message.content}", 
        source="user"
    )
    final_result = await editor.on_messages([edit_message])
    
    return final_result.chat_message.content

# Example usage (uncomment to run):
# result = await sequential_workflow("The impact of AI on healthcare")
# print(result)

### Group Chat Collaboration

In [ ]:
# Create a group chat with all agents
team_chat = RoundRobinGroupChat([researcher, writer, editor])

async def group_collaboration(topic):
    # All agents collaborate on the topic
    message = TextMessage(
        content=f"Let's collaborate on creating content about: {topic}. Each agent should contribute their expertise.",
        source="user"
    )
    
    # Run the group chat for a few rounds
    result = await team_chat.run(task=message)
    return result

# Example usage (uncomment to run):
# result = await group_collaboration("Sustainable energy solutions")
# print(result)

### Dynamic Agent Creation (Based on AutoGen Core)

This shows how to create agents dynamically like in the existing `5_autogen/world.py` example:

In [ ]:
# Example of dynamic agent creation pattern
def create_specialized_agent(specialty, name):
    """Create an agent with a specific specialty"""
    specialties = {
        "finance": "You are a financial analyst. Focus on market trends, investment advice, and economic analysis.",
        "technology": "You are a technology expert. Focus on software development, AI, and tech innovation.",
        "marketing": "You are a marketing specialist. Focus on branding, customer engagement, and growth strategies.",
        "healthcare": "You are a healthcare professional. Focus on medical advances, patient care, and health policy."
    }
    
    system_message = specialties.get(specialty, "You are a general purpose assistant.")
    
    return AssistantAgent(
        name=name,
        model_client=model_client,
        system_message=system_message
    )

# Create multiple specialized agents
agents = {}
specialties = ["finance", "technology", "marketing", "healthcare"]

for i, specialty in enumerate(specialties):
    agent_name = f"{specialty}_agent_{i+1}"
    agents[agent_name] = create_specialized_agent(specialty, agent_name)
    print(f"Created {agent_name} with {specialty} specialty")

print(f"\nTotal agents created: {len(agents)}")

## 2. CrewAI - Team-Based Collaboration

CrewAI excels at creating structured teams with defined roles and processes.

### Basic Crew Setup

In [ ]:
# Note: This example shows the structure - actual CrewAI code would be in a separate project
# See the existing examples in 3_crew/ directory

crew_example = """
from crewai import Agent, Crew, Process, Task

# Create multiple agents with different roles
research_agent = Agent(
    role='Research Specialist',
    goal='Conduct thorough research on given topics',
    backstory='Expert researcher with deep analytical skills',
    verbose=True
)

content_creator = Agent(
    role='Content Creator', 
    goal='Create engaging content based on research',
    backstory='Creative writer with expertise in storytelling',
    verbose=True
)

quality_reviewer = Agent(
    role='Quality Reviewer',
    goal='Review and improve content quality',
    backstory='Experienced editor focused on excellence',
    verbose=True
)

# Define tasks for each agent
research_task = Task(
    description='Research the given topic thoroughly',
    agent=research_agent,
    expected_output='Comprehensive research report'
)

content_task = Task(
    description='Create content based on research',
    agent=content_creator,
    expected_output='Well-written article or content piece'
)

review_task = Task(
    description='Review and refine the content',
    agent=quality_reviewer,
    expected_output='Polished, high-quality final content'
)

# Create the crew
content_crew = Crew(
    agents=[research_agent, content_creator, quality_reviewer],
    tasks=[research_task, content_task, review_task],
    process=Process.sequential,  # Can also be hierarchical
    verbose=True
)

# Run the crew
result = content_crew.kickoff({'topic': 'AI in Education'})
"""

print("CrewAI Multi-Agent Pattern:")
print(crew_example)

### CrewAI Coordination Patterns

In [ ]:
coordination_patterns = """
CrewAI supports different coordination patterns:

1. SEQUENTIAL PROCESS:
   - Agents work one after another
   - Output of one agent becomes input for the next
   - Example: Research → Write → Edit → Publish
   
   process=Process.sequential

2. HIERARCHICAL PROCESS:
   - Manager agent coordinates worker agents
   - Manager delegates tasks and reviews results
   - Example: Project Manager → Developers, Designers, Testers
   
   process=Process.hierarchical
   manager_agent=manager_agent

3. PARALLEL WORKFLOWS (Multiple Crews):
   - Create separate crews for different workstreams
   - Combine results at the end
   
   crew_a = Crew(agents=[...], tasks=[...], process=Process.sequential)
   crew_b = Crew(agents=[...], tasks=[...], process=Process.sequential)
   
   result_a = crew_a.kickoff(input_data)
   result_b = crew_b.kickoff(input_data)
   
   combined_result = combine_results(result_a, result_b)
"""

print(coordination_patterns)

## 3. LangGraph - Graph-Based Agent Orchestration

LangGraph allows you to create complex agent workflows using graph structures.

In [ ]:
# Multi-agent LangGraph pattern
langgraph_example = """
from typing import Annotated, List, Any
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage

# Define shared state
class MultiAgentState(TypedDict):
    messages: Annotated[List[Any], add_messages]
    research_complete: bool
    writing_complete: bool
    review_complete: bool
    current_agent: str

# Create different agent functions
def research_agent(state: MultiAgentState):
    llm = ChatOpenAI(model="gpt-4o-mini")
    system_msg = SystemMessage(content="You are a research agent. Focus on gathering information.")
    
    messages = [system_msg] + state["messages"]
    response = llm.invoke(messages)
    
    return {
        "messages": [response],
        "research_complete": True,
        "current_agent": "research"
    }

def writing_agent(state: MultiAgentState):
    llm = ChatOpenAI(model="gpt-4o-mini")
    system_msg = SystemMessage(content="You are a writing agent. Create content based on research.")
    
    messages = [system_msg] + state["messages"]
    response = llm.invoke(messages)
    
    return {
        "messages": [response],
        "writing_complete": True,
        "current_agent": "writing"
    }

def review_agent(state: MultiAgentState):
    llm = ChatOpenAI(model="gpt-4o-mini")
    system_msg = SystemMessage(content="You are a review agent. Improve and finalize content.")
    
    messages = [system_msg] + state["messages"]
    response = llm.invoke(messages)
    
    return {
        "messages": [response],
        "review_complete": True,
        "current_agent": "review"
    }

# Route between agents
def route_agents(state: MultiAgentState):
    if not state.get("research_complete", False):
        return "research"
    elif not state.get("writing_complete", False):
        return "writing"
    elif not state.get("review_complete", False):
        return "review"
    else:
        return "END"

# Build the graph
graph_builder = StateGraph(MultiAgentState)

# Add agent nodes
graph_builder.add_node("research", research_agent)
graph_builder.add_node("writing", writing_agent)
graph_builder.add_node("review", review_agent)

# Add routing logic
graph_builder.add_conditional_edges(
    START,
    route_agents,
    {"research": "research", "writing": "writing", "review": "review", "END": END}
)

graph_builder.add_conditional_edges(
    "research",
    route_agents,
    {"writing": "writing", "END": END}
)

graph_builder.add_conditional_edges(
    "writing",
    route_agents,
    {"review": "review", "END": END}
)

graph_builder.add_edge("review", END)

# Compile and use
multi_agent_graph = graph_builder.compile()

# Run the multi-agent workflow
initial_state = {
    "messages": [{"role": "user", "content": "Create an article about renewable energy"}],
    "research_complete": False,
    "writing_complete": False,
    "review_complete": False,
    "current_agent": None
}

result = multi_agent_graph.invoke(initial_state)
"""

print("LangGraph Multi-Agent Pattern:")
print(langgraph_example)

## 4. OpenAI Agents SDK - Structured Workflows

The OpenAI Agents SDK provides structured approaches to multi-agent coordination.

In [ ]:
# OpenAI Agents SDK multi-agent pattern
openai_example = """
from openai import OpenAI
import asyncio

class MultiAgentOrchestrator:
    def __init__(self):
        self.client = OpenAI()
        self.agents = {}
        
    def create_agent(self, name, instructions, tools=None):
        """Create a specialized agent"""
        agent = self.client.beta.assistants.create(
            name=name,
            instructions=instructions,
            model="gpt-4o-mini",
            tools=tools or []
        )
        self.agents[name] = agent
        return agent
    
    async def run_sequential_workflow(self, input_message, agent_sequence):
        """Run agents sequentially"""
        current_message = input_message
        
        for agent_name in agent_sequence:
            agent = self.agents[agent_name]
            
            # Create thread and run
            thread = self.client.beta.threads.create()
            
            self.client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=current_message
            )
            
            run = self.client.beta.threads.runs.create(
                thread_id=thread.id,
                assistant_id=agent.id
            )
            
            # Wait for completion
            while run.status not in ['completed', 'failed']:
                await asyncio.sleep(1)
                run = self.client.beta.threads.runs.retrieve(
                    thread_id=thread.id,
                    run_id=run.id
                )
            
            # Get response
            messages = self.client.beta.threads.messages.list(
                thread_id=thread.id
            )
            
            current_message = messages.data[0].content[0].text.value
        
        return current_message
    
    async def run_parallel_workflow(self, input_message, agent_names):
        """Run agents in parallel"""
        tasks = []
        
        for agent_name in agent_names:
            task = self.run_single_agent(agent_name, input_message)
            tasks.append(task)
        
        results = await asyncio.gather(*tasks)
        return results
    
    async def run_single_agent(self, agent_name, message):
        """Run a single agent"""
        agent = self.agents[agent_name]
        
        thread = self.client.beta.threads.create()
        self.client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=message
        )
        
        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        
        while run.status not in ['completed', 'failed']:
            await asyncio.sleep(1)
            run = self.client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
        
        messages = self.client.beta.threads.messages.list(thread_id=thread.id)
        return messages.data[0].content[0].text.value

# Usage example
orchestrator = MultiAgentOrchestrator()

# Create specialized agents
orchestrator.create_agent(
    "researcher",
    "You are a research specialist. Gather comprehensive information on topics."
)

orchestrator.create_agent(
    "writer", 
    "You are a content writer. Create engaging articles based on research."
)

orchestrator.create_agent(
    "editor",
    "You are an editor. Review and improve content for clarity and quality."
)

# Sequential workflow
result = await orchestrator.run_sequential_workflow(
    "Create an article about sustainable technology",
    ["researcher", "writer", "editor"]
)

# Parallel workflow
parallel_results = await orchestrator.run_parallel_workflow(
    "Analyze the topic from different perspectives",
    ["researcher", "writer", "editor"]
)
"""

print("OpenAI Agents SDK Multi-Agent Pattern:")
print(openai_example)

## Best Practices for Multi-Agent Systems

### 1. Clear Role Definition
- Give each agent a specific, well-defined role
- Avoid overlapping responsibilities
- Define clear inputs and expected outputs

### 2. Communication Protocols
- Establish how agents share information
- Define message formats and handoff points
- Handle failures and retries gracefully

### 3. State Management
- Track progress through multi-agent workflows
- Maintain shared context when needed
- Implement checkpointing for long-running processes

### 4. Quality Control
- Add review and validation steps
- Implement feedback loops
- Monitor agent performance and outputs

### 5. Scalability Considerations
- Design for horizontal scaling
- Implement proper error handling
- Consider resource constraints and costs

## Practical Examples by Use Case

### Content Creation Pipeline
```
Research Agent → Content Writer → Editor → SEO Optimizer → Publisher
```

### Software Development Team
```
Product Manager → Architect → Developer → Tester → DevOps → Deployer
```

### Business Analysis
```
Data Collector → Analyst → Modeler → Validator → Report Generator
```

### Customer Support
```
Triage Agent → Specialist Agents (Technical, Billing, General) → Escalation Agent
```

## Framework Comparison

| Framework | Best For | Coordination | Complexity |
|-----------|----------|--------------|------------|
| **AutoGen** | Dynamic agent creation, flexible conversations | Group chats, round-robin | Medium |
| **CrewAI** | Structured team workflows, role-based tasks | Sequential, hierarchical | Low |
| **LangGraph** | Complex workflows, conditional logic | Graph-based routing | High |
| **OpenAI SDK** | Production deployments, structured APIs | Custom orchestration | Medium |

## Getting Started

1. **Start Simple**: Begin with 2-3 agents in a sequential workflow
2. **Choose Your Framework**: Pick based on your specific needs and complexity
3. **Define Clear Roles**: Make sure each agent has a distinct purpose
4. **Test Incrementally**: Add agents and complexity gradually
5. **Monitor and Iterate**: Track performance and refine your approach

## Next Steps

- Explore the existing examples in each framework directory
- Try implementing one of the patterns above
- Experiment with different coordination strategies
- Build your own multi-agent application!

For more detailed examples, check out:
- `5_autogen/world.py` - Dynamic agent creation
- `3_crew/debate/` - CrewAI team collaboration
- `4_langgraph/sidekick.py` - LangGraph patterns
- `2_openai/deep_research/` - OpenAI SDK workflows